In [ ]:
import pandas as pd
import numpy as np

from PIL import Image, ImageOps

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from skimage.filters import sobel
from skimage.io import imread
from skimage import morphology
from skimage.feature import canny
from scipy import ndimage as ndi
from skimage.segmentation import find_boundaries, mark_boundaries, slic

In [ ]:
locations = pd.read_hdf("data/backup/complete_paths.h5", "data")

In [ ]:
locations = locations[((locations['ice'] > 60) & (locations['ice'] <= 100))]# | (locations["ice"] == 255)]

In [ ]:
def get_conc(image):
    try:
        im = np.array(equalize(Image.open(image).convert("L")))[375:800, :]
        #print(np.mean(im))

        # bright enough?
        if np.mean(im) > 70:
            elevation_map = sobel(im)

            hist = np.histogram(im, bins=np.arange(0, 256), normed=True)
            filtered_hist = hist[1][61:][(hist[0][60:] > 0.005)]

            # find filtering borders in the brightness
            low_ice = np.min(filtered_hist)
            high_ice = np.max(filtered_hist)
            low_water = np.floor(low_ice/10)*10
            high_water = np.ceil(high_ice/10)*10

            #print(low_ice, high_ice, low_water, high_water)

            markers = np.zeros_like(im)
            markers[(im < low_water) | (im > high_water)] = 1 # ship, water
            markers[(im > low_ice) & (im < high_ice)] = 3

            segmentation = morphology.watershed(elevation_map, markers)
            return (segmentation == 3).sum()/(im.shape[0]*im.shape[1])
        else:
            return 0.0
    except:
        return np.nan

In [ ]:
test_set = locations[:3000]
test_set["predicted"] = test_set["image"].apply(get_conc)
#test_set

In [ ]:
((test_set["ice"]/100 - test_set["predicted"]) ** 2).mean()

In [ ]:
def equalize(im):
    h = im.convert("L").histogram()
    lut = []
    for b in range(0, len(h), 256):
        # step size
        step = np.sum(h[b:b+256]) / 255
        # create equalization lookup table
        n = 0
        for i in range(256):
            lut.append(n / step)
            n = n + h[i+b]
    # map image through lookup table
    return im.point(lut)

In [ ]:
#test_image = "data/images/remote/20111120-2101.jpeg"
#test_image = "data/images/remote/20150908-2001.jpeg"
test_image = "data/images/remote/20150823-1101.jpeg"

In [ ]:
Image.open(test_image).convert("L")

In [ ]:
im = np.array(equalize(Image.open(test_image).convert("L")))[375:800, :]
elevation_map = sobel(im)
#elevation_map[elevation_map < 10] = 0

markers = np.zeros_like(im)
markers[(im < 60) | (im > 230)] = 1 # ship, water, 86
markers[(im > 61) & (im < 230)] = 3
#markers[(im < 100)] = 1 # ship, water, 98
#markers[(im > 100)] = 3

segmentation = morphology.watershed(elevation_map, markers)
#segmentation = slic(elevation_map, enforce_connectivity=False, sigma=5, n_segments=10)
(segmentation == 3).sum()/(im.shape[0]*im.shape[1])

In [ ]:
background = Image.open(test_image).crop((0,375,1280,800)).convert("RGBA")
im2 = Image.fromarray(elevation_map/np.max(elevation_map)*255).convert("L")

layer = Image.new('RGBA', background.size, color=(120,255,0,255))

Image.composite(background, layer, ImageOps.invert(im2))

In [ ]:
Image.fromarray(segmentation.astype(np.uint8)*40).convert("L")

In [ ]:
hist = np.histogram(im, bins=np.arange(0, 256), normed=True)
plt.plot(hist[1][:-1], hist[0], lw=2) 

In [ ]:
vec = np.convolve(hist[0], np.ones((20,))/20, mode='valid')
plt.plot(vec, lw=2)

In [ ]:
fill_ice = ndi.binary_fill_holes(segmentation-1)
#bounds = find_boundaries(fill_ice)
bounds = mark_boundaries(im, fill_ice)
Image.fromarray(fill_ice.astype(np.uint8)*255)

In [ ]:
filtered_hist = hist[1][61:][(hist[0][60:] > 0.005)]
low_ice = np.min(filtered_hist)
high_ice = np.max(filtered_hist)
low_water = np.floor(low_ice/10)*10
high_water = np.ceil(high_ice/10)*10
print(low_ice, high_ice, low_water, high_water)

In [ ]:
hist[1][(hist[0] < 0.1*np.mean(hist[0])) & (hist[1][1:] > 150)][0]

In [ ]:
(hist[0] > (0.1*np.amax(hist[0]))).sum()

In [ ]:
 np.mean(hist[0])

In [ ]:
locations[locations.index.strftime("%Y%m%d") == "20111120"]